In [1]:
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
import math
import pandas as pd

marathon_2015_2017 = pd.read_csv("./marathon_2015_2017.csv")

In [2]:
# 읽어온 데이터에서 성별, 나이, 페이스, 10K, 20K, 30K, 최종기록을 필터링
# 최종기록으로 정렬 - record 변수에 저장
record = pd.DataFrame(marathon_2015_2017, columns= ['M/F', 'Age', 'Pace', \
           '10K', '20K', '30K', 'Official Time']).sort_values(by=['Official Time'])

# 성별의 데이터릴 'M', 'F'에서 0과 1로 변환
record['M/F'] = record['M/F'].map({'M':1, 'F':0})

# 데이터 프레임을 리스트로 변환
record_list = record.values.tolist()

# gender_list 생성
gender_list = ['Femail','Mail']

# 버튼이 클릭되면 화면에 표시될 차트를 생성하여 구색을 갖추고,
# gn변수에 등록해두어, 차트 표시명령 시 세트명령을 사용하도록 준비합니다
grad_fig = Figure( figsize=(10,4), dpi=100)
grad_ax = grad_fig.add_subplot(111)
grad_ax.set_xlim(15,88)
grad_ax.set_ylim(0,1300)
grad_ax.set_xlabel("Age : Age on race day")
grad_ax.set_ylabel("Pace : Runner's overall minute per mile pace")

g_xdata, g_ydata = [], []
gn, =grad_ax.plot([], [], 'ro')

In [3]:
def histogram():
    # t_gCbbox.get()의 값은 선택된 'Male' 또는 'Female'
    gender = t_gCbbox.get()
    # gender_list.index(gender) 선택된 값을 gender_list의 인덱스를 이용해서 0 또는 1로 변환
    t_g = int(gender_list.index(gender))
    
    # 나머지 스핀박스 값 추출
    t_a = int(t_aSpbox.get())  # 나이
    t_p = int(t_pSpbox.get())  # 페이스
    
    # 차트에 표시할 성별에 따른 점 색깔 설정
    if(t_g):
        gender_color = 'b'
    else:
        gender_color = 'r'
    
    # 선택된 성별에 따라 필터링
    gender_record = record[ record['M/F'] == t_g]
    # 선택된 나이에 따라서 필터링
    gender_age_record = gender_record[ gender_record.Age == t_a ]
    # 필터링된 데이터들을 모두 리스트로 변환
    gender_age_record_list = gender_age_record.values.tolist()
    
    # 성별에 따른 색깔의 점으로 같은 연령별(x축) 페이스들(y축)을 표시
    grad_ax.plot(gender_record.Age, gender_record.Pace, '.', \
                                         color = gender_color, alpha=0.3)
    # 선택된 연령과 페이스로 노란색 다이아몬드 표시
    grad_ax.plot(t_a, t_p, 'yd')
    # 나이로 필터링 된 데이터에서 페이스에 대한 기술 통계값 계산
    stat = gender_age_record['Pace'].describe()
    
    # 차트 타이틀 제작
    title = 'Gender : '+gender_list[t_g]+', Age :'+str(t_a)
    grad_ax.set_title(title)
    
    # 오른쪽 상단 위치에 annotate로 인원, 사분위 값들을 차례로 표시
    grad_ax.annotate("%10s %7i"%('Count : ', stat[0]), (75, 1200), fontsize=10)
    grad_ax.annotate("%10s %7.3f"%('Mean : ', stat[1]), (75, 1150), fontsize=10)
    grad_ax.annotate("%10s %7.3f"%('25% : ', stat[3]), (75, 1100), fontsize=10)
    grad_ax.annotate("%10s %7.3f"%('75% : ', stat[5]), (75, 1050), fontsize=10)
    grad_fig.canvas.draw()

In [4]:
def seconds_to_hhmmss(seconds):
    hours = seconds // (60*60)
    seconds %= (60*60)
    minutes = seconds // 60
    seconds %= 60
    return "%02i:%02i%02i" % (hours, minutes, seconds)

In [5]:
def learning():
    import tensorflow as tf
    import numpy as np
    # 성별 나이 페이스 추출
    gender = t_gCbbox.get()
    t_g = int(gender_list.index(gender))  
    t_a = int(t_aSpbox.get())  # 대상 데이터의 순위
    t_p = int(t_pSpbox.get())
    
    # train 횟수, learning rate 추출
    t_t = int(t_tSpbox.get())  
    t_r = float(t_rSpbox.get())  
    
    # 성별 나이 페이스를 feature로, 최종기록을 target으로 분리
    x_train = [ r[0:3] for r in record_list ]
    y_train = [ r[-1] for r in record_list ]
    
    # Sequential model 생성
    model = tf.keras.models.Sequential()
    # 입력 값 종류 3개, 출력 1개
    model.add(tf.keras.layers.Dense(1, input_shape=(3,)))
    # 최적화 도구로 sgd
    sgd = tf.keras.optimizers.SGD(lr=t_r)
    # 평가도구 mse로 설정하고, 위 사항으로 모델을 완성
    model.compile(loss='mse', optimizer=sgd)
    # 학습 전에 컴파일된 모델을 요약하여 출력
    model.summary()
    # 학습
    history = model.fit(np.array(x_train), np.array(y_train), epochs=t_t)
    
    # 예측값 및 중간 코스트값 출력을 위한 제목들
    log_ScrolledText.insert(END, '\nGender:'+gender_list[t_g]+'\, Age:'+str(t_a)+\
                           ', Pace:'+str(t_p)+'\n', 'TITLE')
    log_ScrolledText.insert(END, '\n\nCost Decent\n\n', 'HEADER')
    log_ScrolledText.insert(END, '%20s %20s' % ('Step', 'Cost')+'\n\n')
    
    # 100회 학습마다 코스트 값 출력
    for step in range(t_t):
        if step % 100 == 0:
            cost_val = history.history['loss'][step]
            log_ScrolledText.insert(END, "%20i %20.5f" % (step, cost_val)+'\n')
    
    # predict 예측(선택한 성별 나이 페이스로 최종 기록 예측)
    winner = [t_g, t_a, t_p]
    time = model.predict(np.array([winner]))
    # 시분초 형식의 예측기록과 초단위의 예측 기록을 한 번에 출력하기 위한 텍스트 구성
    ml_time = seconds_to_hhmmss(time[0][0])+ '('+str(time[0][0])+')'
    # 예측값 출력을 위한 제목
    log_ScrolledText.insert(END, "%20s" % ('\n\nThe Prediction Recoreds\n\n'), \
                            'HEADER')
    # 출력 내용의 열 제목들
    log_ScrolledText.insert(END, "%10s %10s %10s %50s" % ('Gender','Age','Pace',\
                                    'Record Prediction(Second) at 42.195km')+'\n\n')
    # 예측값 출력
    log_ScrolledText.insert(END, "%10s %10s %10s %50s" % (gender_list[t_g], str(t_a), \
                                                         str(t_p), ml_time)+'\n')

In [ ]:
# 메인윈도우 생성 : 타이틀 - Multi Variable Matrix Linear Regression
main = Tk()
main.title("Multi Variable Matrix Linear Regression")
main.geometry()

# 라벨생성 - 라벨 텍스트 : Multi Variable Matrix Linear Regression
# 굴림체, 18포인트, row=0, column=0, columnspan=6
label = Label(main, text='Multi Variable Matrix Linear Regression')
label.config(font=("굴림", 18))
label.grid(row=0, column=0, columnspan=6)

# 콤보상자 생성 - 콤보상자 내 item : gender_list, 위치( 1(row), 0(column) )
t_gVal = StringVar(value=gender_list[0]) # 연결된 변수 값 - 최초표시값을 저장
t_gCbbox = ttk.Combobox(main, textvariable=t_gVal)
t_gCbbox['values'] = gender_list  # 전체 보유 item설정
t_gCbbox.config(state='readonly')
t_gCbbox.grid(row=1, column=1)

# 라벨생성 텍스트 : Gender # row=1, column=0
t_gLabel = Label(main, text='Gender : ')
t_gLabel.grid(row=1, column=0)

# 스핀박스 생성
# 초기값 45, 시작값 18, 끝 값 84, 증가량 1, 오른쪽정렬, 스핀박스버튼 외 수정불가
# row=1, column=3
# 라벨 생성 # 텍스트 Age : row=1, column=4
t_aVal = IntVar(value=45)
t_aSpbox = Spinbox(main, textvariable=t_aVal, from_=18, to=84, increment=1, \
                                                                       justify=RIGHT)
t_aSpbox.config(state='readonly')
t_aSpbox.grid(row=1, column=3)
t_aLabel=Label(main, text='Age : ')
t_aLabel.grid(row=1, column=4)

# 스핀박스2 생성
# 초기값 500, 시작값 0, 끝 값 1500, 증가량 1, 오른쪽 정렬, 스핀박스버튼 외 수정불가
# row=1, column=5
# 라벨 생성 # 텍스트 Pace : row=1, column=4
t_pVal = IntVar(value=500)
t_pSpbox = Spinbox(main, textvariable=t_pVal, from_=0, to=1500, increment=1, \
                                                                       justify=RIGHT)
t_pSpbox.config(state='readonly')
t_pSpbox.grid(row=1, column=5)
t_pLabel=Label(main, text='Pace : ')
t_pLabel.grid(row=1, column=4)

# 스핀박스3 생성
# 초기값 2000, 시작값 0, 끝 값 100000, 증가량 1000, 오른쪽 정렬,
# row=2, column=1
# 라벨생성 #텍스트 Number of train : row=2, column=0
t_tVal = IntVar(value=2000)
t_tSpbox = Spinbox(main, textvariable=t_tVal, from_=0, to=100000, increment=1000, \
                                                                       justify=RIGHT)
t_tSpbox.grid(row=2, column=1)
t_tLabel=Label(main, text='Number of train : ')
t_tLabel.grid(row=2, column=0)

# 스핀박스4 생성
# 초기값 0.000001, 시작값 0, 끝 값 1, 증가량 0.000001, 오른쪽 정렬,
# row=2, column=3
# 라벨 생성 # 텍스트 Learning rate : row=2, column=2
t_rVal = DoubleVar(value=1e-6)
t_rSpbox = Spinbox(main, textvariable=t_rVal, from_=0, to=1, increment=(1e-6), \
                                                                       justify=RIGHT)
t_rSpbox.grid(row=2, column=3)
t_rLabel=Label(main, text='Learning rate : ')
t_rLabel.grid(row=2, column=2)

# 버튼 생성 
btn1 = Button(main, text="Histogram", height=2,command=lambda:histogram())
btn1.grid(row=2, column=4, columnspan=1, sticky=(W,E))
btn2 = Button(main, text="Prediction", height=2,command=lambda:learning())
btn2.grid(row=2, column=5, columnspan=1, sticky=(W,E))

# 차트 캔버스
grad_canvas = FigureCanvasTkAgg(grad_fig, main)
grad_canvas.get_tk_widget().grid(row=3, column=0, columnspan=6)

# Scrolled Text
log_ScrolledText = tkst.ScrolledText(main,height=15)
log_ScrolledText.grid(row=4, column=0, columnspan=6, sticky=(N, S, W, E))
log_ScrolledText.configure(font='굴림')
log_ScrolledText.tag_config('RESULT', foreground='blue', font=("굴림", 12))
log_ScrolledText.tag_config('HEADER', foreground='black', font=("굴림", 14))
log_ScrolledText.tag_config('TITLE', foreground='red', font=("굴림", 18))

# 어제 데이터 : [5,10,15,20,21.098,25,30,35]과 [321, 758, 1470, 2145 ... 5021]
# 오늘 데이터세트를 입력하고 특정 연령, 성별의 최종기록 예측


main.mainloop()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
2489/2489 [==============================] - 2s 784us/step - loss: 721464.7980
Epoch 2/2000
2489/2489 [==============================] - 1s 467us/step - loss: 8194.1596
Epoch 3/2000
2489/2489 [==============================] - 1s 455us/step - loss: 6334.3134
Epoch 4/2000
2489/2489 [==============================] - 1s 447us/step - loss: 1468.8626
Epoch 5/2000
2489/2489 [==============================] - 1s 449us/step - loss: 4394.3922
Epoch 6/2000
2489/2489 [==============================] - 1s 456us/step - loss: 1810.2945
Epoch 7/2000
2489/2489 [==============================] - 1s 456us/step - loss: 5251.5187
Epoch 8/2000
2489/2489 [

Epoch 86/2000
2489/2489 [==============================] - 1s 445us/step - loss: 6578.8377
Epoch 87/2000
2489/2489 [==============================] - 1s 452us/step - loss: 6258.1598
Epoch 88/2000
2489/2489 [==============================] - 1s 443us/step - loss: 2781.0156
Epoch 89/2000
2489/2489 [==============================] - 1s 443us/step - loss: 747.5891
Epoch 90/2000
2489/2489 [==============================] - 1s 434us/step - loss: 461.7140
Epoch 91/2000
2489/2489 [==============================] - 1s 443us/step - loss: 750.9097
Epoch 92/2000
2489/2489 [==============================] - 1s 443us/step - loss: 5762.5699
Epoch 93/2000
2489/2489 [==============================] - 1s 449us/step - loss: 1959.2158
Epoch 94/2000
2489/2489 [==============================] - 1s 439us/step - loss: 201.0196
Epoch 95/2000
2489/2489 [==============================] - 1s 440us/step - loss: 2901.8934
Epoch 96/2000
2489/2489 [==============================] - 1s 439us/step - loss: 9810.4093
Epo

2489/2489 [==============================] - 1s 441us/step - loss: 467.8478
Epoch 176/2000
2489/2489 [==============================] - 1s 439us/step - loss: 325.5735
Epoch 177/2000
2489/2489 [==============================] - 1s 438us/step - loss: 5097.5193
Epoch 178/2000
2489/2489 [==============================] - 1s 454us/step - loss: 3479.8054
Epoch 179/2000
2489/2489 [==============================] - 1s 447us/step - loss: 19103.5277
Epoch 180/2000
2489/2489 [==============================] - 1s 442us/step - loss: 7816.6550
Epoch 181/2000
2489/2489 [==============================] - 1s 444us/step - loss: 6652.4974
Epoch 182/2000
2489/2489 [==============================] - 1s 439us/step - loss: 203.4922
Epoch 183/2000
2489/2489 [==============================] - 1s 447us/step - loss: 4239.3466
Epoch 184/2000
2489/2489 [==============================] - 1s 443us/step - loss: 740.2949
Epoch 185/2000
2489/2489 [==============================] - 1s 437us/step - loss: 127.2416
Epoch 1

2489/2489 [==============================] - 1s 448us/step - loss: 4034.9570
Epoch 265/2000
2489/2489 [==============================] - 1s 449us/step - loss: 3870.2805
Epoch 266/2000
2489/2489 [==============================] - 1s 444us/step - loss: 11668.7524
Epoch 267/2000
2489/2489 [==============================] - 1s 445us/step - loss: 4540.9399
Epoch 268/2000
2489/2489 [==============================] - 1s 450us/step - loss: 4098.6426
Epoch 269/2000
2489/2489 [==============================] - 1s 454us/step - loss: 699.5915
Epoch 270/2000
2489/2489 [==============================] - 1s 444us/step - loss: 8405.7638
Epoch 271/2000
2489/2489 [==============================] - 1s 452us/step - loss: 7293.9103
Epoch 272/2000
2489/2489 [==============================] - 1s 444us/step - loss: 4998.9687
Epoch 273/2000
2489/2489 [==============================] - 1s 451us/step - loss: 180.2286
Epoch 274/2000
2489/2489 [==============================] - 1s 449us/step - loss: 2660.7679
Epoc